<center><h1>WRN: Cifar10</h1></center>

## Imports

In [29]:
from __future__ import division,print_function

# %matplotlib inline
# %load_ext autoreload
# %autoreload 2

import sys, os
from tqdm import tqdm_notebook as tqdm

import random
import matplotlib.pyplot as plt
import math

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable, grad
from torchvision import datasets, transforms
from torch.nn.parameter import Parameter
from torch.distributions import Categorical

from pgd import PGD, PGD_l2

import calculate_log as callog

import warnings
warnings.filterwarnings('ignore')

In [30]:
torch.cuda.set_device(7)

## Model definition

In [31]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes,track_running_stats=True)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes,track_running_stats=True)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                               padding=0, bias=False) or None
    def forward(self, x):
        if not self.equalInOut:
            x = self.relu1(self.bn1(x))
            torch_model.record(x)
        else:
            out = self.relu1(self.bn1(x))
            torch_model.record(out)
        if self.equalInOut:
            out = self.conv1(out)
        else:
            out = self.conv1(x)
        torch_model.record(out)
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        t = self.relu2(self.bn2(out))
        torch_model.record(t)
        out = self.conv2(t)
        torch_model.record(out)
        if not self.equalInOut:
            return torch.add(self.convShortcut(x), out)
        else:
            return torch.add(x, out)

class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate)
    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, dropRate):
        layers = []
        for i in range(nb_layers):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)

class WideResNet(nn.Module):
    def __init__(self, depth, num_classes, widen_factor=1, dropRate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert((depth - 4) % 6 == 0)
        self.collecting = False
        n = (depth - 4) // 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, dropRate)
        # 2nd block
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, dropRate)
        # 3rd block
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, dropRate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(nChannels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    
    def forward(self, x):
        out = self.conv1(x)
        torch_model.record(out)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        lg = self.fc(out)
        return lg
    
    def record(self, t):
        if self.collecting:
            self.gram_feats.append(t)
    
    def gram_forward(self, x):
        self.collecting = True
        self.gram_feats = []
        logits = self.forward(x)
        self.collecting = False
        temp = self.gram_feats
        self.gram_feats = []
        return logits, temp
    
    def gram_feature_list(self,x):
        self.collecting = True
        self.gram_feats = []
        self.forward(x)
        self.collecting = False
        temp = self.gram_feats
        self.gram_feats = []
        return temp
    
    def load(self, path="cifar10wrn_baseline_epoch_99.pt"):
#     def load(self, path='cifar10_wrn_oe_scratch_epoch_99.pt'):
        tm = torch.load(path,map_location="cpu")        
        self.load_state_dict(tm,strict=False)
    
    def get_min_max(self, data, power):
        mins = []
        maxs = []
        
        for i in range(0,len(data),64):
            batch = data[i:i+64].cuda()
            feat_list = self.gram_feature_list(batch)
            for L,feat_L in enumerate(feat_list):
                    
                if L==len(mins):
                    mins.append([None]*len(power))
                    maxs.append([None]*len(power))
                
                for p,P in enumerate(power):
                    g_p = G_p(feat_L,P)
                    
                    current_min = g_p.min(dim=0,keepdim=True)[0]
                    current_max = g_p.max(dim=0,keepdim=True)[0]
                    
                    if mins[L][p] is None:
                        mins[L][p] = current_min
                        maxs[L][p] = current_max
                    else:
                        mins[L][p] = torch.min(current_min,mins[L][p])
                        maxs[L][p] = torch.max(current_max,maxs[L][p])
                                
        return mins,maxs
    
    def get_grams(self,data,powers):
        grams = {}
        for i in range(0,len(data),64):            
            batch = data[i:i+64].cuda()
            
            feat_list = self.gram_feature_list(batch)
            batch_deviations = []
            
            for L,feat_L in enumerate(feat_list):
                if L not in grams:
                    grams[L] = {}
                for p, P in enumerate(powers):
                    g_p = G_p(feat_L,P)
                    
                    if p not in grams[L]:
                        grams[L][p] = g_p
                    else:
                        grams[L][p] = torch.cat((grams[L][p], g_p))
                    
        return grams
    
    def get_average(self,data,powers):
        grams = {}
        for i in range(0,len(data),64):            
            batch = data[i:i+64].cuda()
            
            feat_list = self.gram_feature_list(batch)
            batch_deviations = []
            
            for L,feat_L in enumerate(feat_list):
                if L not in grams:
                    grams[L] = {}
                for p, P in enumerate(powers):
                    g_p = G_p(feat_L,P)
                    
                    if p not in grams[L]:
                        grams[L][p] = g_p
                    else:
                        grams[L][p] = torch.cat((grams[L][p], g_p))
                        
        
        for L, ps in grams.items():
            for p in ps:
                grams[L][p] = grams[L][p].mean(dim=0)
                    
        return grams
    
    def get_deviations(self,data,power,mins,maxs, folder="~/gram_screenshots/test_data/"):
        folder = os.path.expanduser(folder)
        if not os.path.exists(folder):
            os.mkdir(folder)
            
        deviations = []
        for i in range(0,len(data),64):            
            batch = data[i:i+64].cuda()
            feat_list = self.gram_feature_list(batch)
            batch_deviations = []
            for L,feat_L in enumerate(feat_list):
                dev = 0
                use_fl = np.random.randint(0,2)
                for p,P in enumerate(power):
                    save_path = ""
                    if i < 1 and use_fl == 1:
                        save_path = folder + "batch_{},feat_layer_{},power_{}.png".format(i, L, p)
                        
                    g_p = G_p(feat_L,P)
                    
                    dev +=  (F.relu(mins[L][p]-g_p)/torch.abs(mins[L][p]+10**-6)).sum(dim=1,keepdim=True)
                    dev +=  (F.relu(g_p-maxs[L][p])/torch.abs(maxs[L][p]+10**-6)).sum(dim=1,keepdim=True)
                batch_deviations.append(dev.cpu().detach().numpy())
            batch_deviations = np.concatenate(batch_deviations,axis=1)
            deviations.append(batch_deviations)
        deviations = np.concatenate(deviations,axis=0)
        
        return deviations

torch_model = WideResNet(depth=40,widen_factor=2, num_classes=10)

torch_model.load()
torch_model.cuda()
torch_model.params = list(torch_model.parameters())
torch_model.eval()
print("Done")    

Done


In [32]:
batch_size = 128
# mean = np.array([[0.4914, 0.4822, 0.4465]]).T

# std = np.array([[0.2023, 0.1994, 0.2010]]).T

# normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

normalize = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
        
    ])
transform_test = transforms.Compose([
        transforms.CenterCrop(size=(32, 32)),
        transforms.ToTensor(),
        normalize
    ])

transform_test_noisy = transforms.Compose([
    transforms.CenterCrop(size=(32, 32)),
        transforms.ToTensor(),
        normalize,
        transforms.Lambda(lambda x: x + torch.randn(x.shape)),
    ])

transform_test_noisy_unif = transforms.Compose([
    transforms.CenterCrop(size=(32, 32)),
        transforms.ToTensor(),
        normalize,
        transforms.Lambda(lambda x: x + torch.from_numpy(np.random.uniform(low=-1, high=1, size=x.shape).astype(np.float32))),
    ])

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=True, download=True,
                   transform=transform_train),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=False, transform=transform_test),
    batch_size=batch_size)
test_loader_noisy = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=False, transform=transform_test_noisy_unif),
    batch_size=batch_size)

data_train = list(torch.utils.data.DataLoader(
        datasets.CIFAR10('data', train=True, download=True,
                       transform=transform_test),
        batch_size=1, shuffle=False))

data = list(torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=False, download=True,
                   transform=transform_test),
    batch_size=1, shuffle=False))

# cifar100 = list(torch.utils.data.DataLoader(
#     datasets.CIFAR100('data', train=False, download=True,
#                    transform=transform_test),
#     batch_size=1, shuffle=True))

# svhn = list(torch.utils.data.DataLoader(
#     datasets.SVHN('data', split="test", download=True,
#                    transform=transform_test),
#     batch_size=1, shuffle=True))

# isun = list(torch.utils.data.DataLoader(
#     datasets.ImageFolder("iSUN/",transform=transform_test),batch_size=1,shuffle=False))

# lsun_c = list(torch.utils.data.DataLoader(
#     datasets.ImageFolder("LSUN/",transform=transform_test),batch_size=1,shuffle=True))

# lsun_r = list(torch.utils.data.DataLoader(
#     datasets.ImageFolder("LSUN_resize/",transform=transform_test),batch_size=1,shuffle=True))

# tinyimagenet_c = list(torch.utils.data.DataLoader(
#     datasets.ImageFolder("Imagenet/",transform=transform_test),batch_size=1,shuffle=True))

# tinyimagenet_r = list(torch.utils.data.DataLoader(
#     datasets.ImageFolder("Imagenet_resize/",transform=transform_test),batch_size=1,shuffle=True))

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [33]:
train_preds = []
train_confs = []
train_logits = []
for idx in range(0,len(data_train),128):
    batch = torch.squeeze(torch.stack([x[0] for x in data_train[idx:idx+128]]),dim=1).cuda()
    
    logits = torch_model(batch)
    confs = F.softmax(logits,dim=1).cpu().detach().numpy()
    preds = np.argmax(confs,axis=1)
    logits = (logits.cpu().detach().numpy())

    train_confs.extend(np.max(confs,axis=1))    
    train_preds.extend(preds)
    train_logits.extend(logits)

test_preds = []
test_confs = []
test_logits = []

for idx in range(0,len(data),128):
    batch = torch.squeeze(torch.stack([x[0] for x in data[idx:idx+128]]),dim=1).cuda()
    
    logits = torch_model(batch)
    confs = F.softmax(logits,dim=1).cpu().detach().numpy()
    preds = np.argmax(confs,axis=1)
    logits = (logits.cpu().detach().numpy())

    test_confs.extend(np.max(confs,axis=1))    
    test_preds.extend(preds)
    test_logits.extend(logits)

def detect(all_test_deviations,all_ood_deviations, test_confs = None, ood_confs=None, verbose=True, normalize=True):
    if test_confs is not None:
        test_confs = np.array(test_confs)
        ood_confs = np.array(ood_confs)
    
    average_results = {}
        
    validation_indices = random.sample(range(len(all_test_deviations)),int(0.1*len(all_test_deviations)))
    test_indices = sorted(list(set(range(len(all_test_deviations)))-set(validation_indices)))

    validation = all_test_deviations[validation_indices]
    test_deviations = all_test_deviations[test_indices]

    t95 = validation.mean(axis=0)+10**-7
    if not normalize:
        t95 = np.ones_like(t95)
    test_deviations = (test_deviations/t95[np.newaxis,:]).sum(axis=1)
    ood_deviations = (all_ood_deviations/t95[np.newaxis,:]).sum(axis=1)
        
#         if test_confs is not None:
#             thresh = np.max((validation/t95[np.newaxis,:]).sum(axis=1))
                        
#             ood_deviations = ood_deviations - threszh*ood_confs

#             test_deviations = test_deviations - thresh*test_confs[test_indices]
    
    results = callog.compute_metric(-test_deviations,-ood_deviations)
    print(results)
    for m in results:
        average_results[m] = average_results.get(m,0)+results[m]

    for m in average_results:
        average_results[m] /= 1
    if verbose:
        callog.print_results(average_results)
    return average_results

def cpu(ob):
    for i in range(len(ob)):
        for j in range(len(ob[i])):
            ob[i][j] = ob[i][j].cpu()
    return ob
    
def cuda(ob):
    for i in range(len(ob)):
        for j in range(len(ob[i])):
            ob[i][j] = ob[i][j].cuda()
    return ob

class Detector:
    def __init__(self):
        self.all_test_deviations = None
        self.mins = {}
        self.maxs = {}
        
        self.classes = range(10)
    
    def compute_minmaxs(self,data_train,POWERS=[10]):
        for PRED in self.classes:
            train_indices = np.where(np.array(train_preds)==PRED)[0]
            train_PRED = torch.squeeze(torch.stack([data_train[i][0] for i in train_indices]),dim=1)
            mins,maxs = torch_model.get_min_max(train_PRED,power=POWERS)
            self.mins[PRED] = cpu(mins)
            self.maxs[PRED] = cpu(maxs)
            torch.cuda.empty_cache()
    
    def get_test_grams(self, POWERS):
        class_avgs = []
        
        for PRED in self.classes:
            test_indices = np.where(np.array(test_preds)==PRED)[0]
            test_PRED = torch.squeeze(torch.stack([data[i][0] for i in test_indices]),dim=1)

            average_grams = torch_model.get_grams(test_PRED, powers=POWERS)
            
            class_avgs.append(average_grams)
            
        return class_avgs
    
    def average_test_gram(self, POWERS):
        class_avgs = []
        
        for PRED in self.classes:
            test_indices = np.where(np.array(test_preds)==PRED)[0]
            test_PRED = torch.squeeze(torch.stack([data[i][0] for i in test_indices]),dim=1)

            average_grams = torch_model.get_average(test_PRED, powers=POWERS)
            
            class_avgs.append(average_grams)
            
        return class_avgs
    
    
    def get_data(self,data,POWERS,msp=False, folder=""):
        ood_preds = []
        ood_confs = []
        
        pred = torch.squeeze(torch.stack([x[0] for x in data]),dim=1).cuda()
        average_grams = torch_model.get_average(pred, powers=POWERS)
        
        return average_grams
    
    def compute_test_deviations(self,POWERS=[10]):
        all_test_deviations = None
        all_test_deviations_msp = None
        all_test_confs = []
        for PRED in self.classes:
            test_indices = np.where(np.array(test_preds)==PRED)[0]
            test_PRED = torch.squeeze(torch.stack([data[i][0] for i in test_indices]),dim=1)
            test_confs_PRED = np.array([test_confs[i] for i in test_indices])
            all_test_confs.extend(test_confs_PRED)
            mins = cuda(self.mins[PRED])
            maxs = cuda(self.maxs[PRED])
            test_deviations = torch_model.get_deviations(test_PRED,power=POWERS,mins=mins,maxs=maxs)
            test_deviations_MSP = test_deviations/test_confs_PRED[:,np.newaxis]
            cpu(mins)
            cpu(maxs)
            if all_test_deviations is None:
                all_test_deviations = test_deviations
                all_test_deviations_MSP = test_deviations_MSP
            else:
                all_test_deviations = np.concatenate([all_test_deviations,test_deviations],axis=0)
                all_test_deviations_MSP = np.concatenate([all_test_deviations_MSP,test_deviations_MSP],axis=0)
            torch.cuda.empty_cache()
        self.all_test_confs = all_test_confs
        self.all_test_deviations = all_test_deviations
        self.all_test_deviations_MSP = all_test_deviations_MSP
 
    def compute_ood_deviations(self,ood,POWERS=[10],msp=False, folder=""):
        ood_preds = []
        ood_confs = []
        
        for idx in range(0,len(ood),128):
            batch = torch.squeeze(torch.stack([x[0] for x in ood[idx:idx+128]]),dim=1).cuda()
            logits = torch_model(batch)
            confs = F.softmax(logits,dim=1).cpu().detach().numpy()
            preds = np.argmax(confs,axis=1)
            
            ood_confs.extend(np.max(confs,axis=1))
            ood_preds.extend(preds)  
            torch.cuda.empty_cache()
            
            
#         print("MSP")
#         callog.print_results(callog.compute_metric(np.array(test_confs),np.array(ood_confs)))
        
        all_ood_deviations = None
        all_ood_deviations_MSP = None
        all_ood_confs = []
        for PRED in self.classes:
            ood_indices = np.where(np.array(ood_preds)==PRED)[0]
            if len(ood_indices)==0:
                continue
            ood_PRED = torch.squeeze(torch.stack([ood[i][0] for i in ood_indices]),dim=1)
            
            ood_confs_PRED =  np.array([ood_confs[i] for i in ood_indices])
            
            all_ood_confs.extend(ood_confs_PRED)
            
            mins = cuda(self.mins[PRED])
            maxs = cuda(self.maxs[PRED])
            ood_deviations = torch_model.get_deviations(ood_PRED,power=POWERS,mins=mins,maxs=maxs)
            ood_deviations_MSP = ood_deviations/ood_confs_PRED[:,np.newaxis]
            cpu(self.mins[PRED])
            cpu(self.maxs[PRED])            
            if all_ood_deviations is None:
                all_ood_deviations = ood_deviations
                all_ood_deviations_MSP = ood_deviations_MSP
            else:
                all_ood_deviations = np.concatenate([all_ood_deviations,ood_deviations],axis=0)
                all_ood_deviations_MSP = np.concatenate([all_ood_deviations_MSP,ood_deviations_MSP],axis=0)
            torch.cuda.empty_cache()
        
        self.all_ood_confs = all_ood_confs
        
        print("Gram")
                
        average_results = detect(self.all_test_deviations,all_ood_deviations)
#         print("Gram+MSP")
#         average_results = detect(self.all_test_deviations,all_ood_deviations,self.all_test_confs,self.all_ood_confs)
        return average_results, self.all_test_deviations, all_ood_deviations
    
def get_b(d):
    batch_size = 128
    bx = []
    by = []
    tens = transforms.ToTensor()
    for idx in range(0,len(d),batch_size):
        bx_batch = torch.squeeze(torch.stack([tens(x[0]) for x in d[idx:idx+batch_size]]),dim=1)
        bx.append(bx_batch)
        by.append(torch.Tensor([x[1] for x in d[idx:idx+batch_size]]).type(torch.LongTensor))
    
    return bx, by

def advs_p(p, bxs, bys):
    advs = []
    for i in range(len(bxs)):
        advs_batch = p(torch_model, bxs[i].cuda(), bys[i].cuda())

        advs.append(advs_batch)

    torch.cuda.empty_cache()
    
    return advs

def pipeline_batch(bxs):
    pil = transforms.ToPILImage()
    return torch.squeeze(torch.stack([transform_test(pil(bx)) for bx in bxs]), dim=1)

def adversarial_acc(advs, bys):
    torch_model.eval()
    correct = 0
    total = 0

    for i in range(len(advs)):
        pipelined = pipeline_batch(advs[i].cpu())

        x = pipelined.cuda()
        y = bys[i].numpy()

        correct += (y==np.argmax(torch_model(x).detach().cpu().numpy(),axis=1)).sum()
        total += y.shape[0]


    print("Adversarial Test Accuracy: ", correct/total)
    
def ds_grouped(bxs, bys):
    ds = []
    for i in range(len(bxs)):
        pipelined = pipeline_batch(bxs[i].cpu())
        for j in range(len(bxs[i])):
            ds.append((pipelined[j], bys[i][j]))
    return ds

def adversarial_scores(advs, bys, powers, folder=""):
    ds = ds_grouped(advs, bys)
    tds = list(torch.utils.data.DataLoader(ds,batch_size=1,shuffle=True))
    _ = detector.compute_ood_deviations(tds, POWERS=powers, folder=folder)
    
def model_accuracy():
    torch_model.eval()
    correct = 0
    total = 0
    for x,y in test_loader:
        x = x.cuda()
        y = y.numpy()
        correct += (y==np.argmax(torch_model(x).detach().cpu().numpy(),axis=1)).sum()
        total += y.shape[0]
    print("Model Test Accuracy: ", correct/total)

In [24]:
# def G_p(ob, p, save_path=""):
#     temp = ob.detach()
#     temp = temp.reshape(temp.shape[0],temp.shape[1],-1)
    
#     temp = ((torch.matmul(temp,temp.transpose(dim0=2,dim1=1))))
    
#     temp = temp.sum(dim=2)
# #     temp = temp.reshape(temp.shape[0],-1)
    
#     return temp.reshape(ob.shape[0], -1)

def G_p(ob, p):
    temp = ob.detach()
    
    temp = temp**p
    temp = temp.reshape(temp.shape[0],temp.shape[1],-1)
    temp = ((torch.matmul(temp,temp.transpose(dim0=2,dim1=1)))).sum(dim=2) 
    temp = (temp.sign()*torch.abs(temp)**(1/p)).reshape(temp.shape[0],-1)
    
    return temp

In [34]:
powers = [1]

print("Initializing Test\n")

print("1. Fitting Gram Detector")
detector = Detector()
detector.compute_minmaxs(data_train,POWERS=powers)

detector.compute_test_deviations(POWERS=powers)

print("2. Generating Adversarial Examples")

cifar10 = list(datasets.CIFAR10('data', train=False))
# print("3. Calculating L2")
# xs, ys = get_b(cifar10)
# pl2 = PGD_l2()
# advs_l2 = advs_p(pl2, xs, ys)

print("4. Calculating L_Inf")
xs, ys = get_b(cifar10)
pinf = PGD(mean=[0.5, .5, .5], std=[.5, .5, .5])
advs_inf = advs_p(pinf, xs, ys)

Initializing Test

1. Fitting Gram Detector
2. Generating Adversarial Examples
4. Calculating L_Inf


In [26]:
xs, ys = get_b(cifar10)

# print("Calculating Scores:\n")

# model_accuracy()

# print("\n**L_2 Adversarial:**\n")
# print("```")
# adversarial_acc(advs_l2, ys)
# adversarial_scores(advs_l2, ys, folder="~/gram_screenshots/L_2_Adversarial/")
# print("```")

print("\n**L_Inf Adversarial:**\n")
print("```")
# adversarial_acc(advs_inf, ys)
adversarial_scores(advs_inf, ys, powers)
print("```")


**L_Inf Adversarial:**

```
Gram
{'AUIN': 0.4253880510901974, 'DTACC': 0.5340333333333334, 'AUOUT': 0.5334489447476696, 'TNR': 0.06059999999999999, 'AUROC': 0.4732171666666667}
 TNR    AUROC  DTACC  AUIN   AUOUT 
  6.060 47.322 53.403 42.539 53.345
```


In [12]:
def advs_p(p, bxs, bys):
    advs = []
    for i in range(len(bxs)):
        advs_batch = p(torch_model, bxs[i].cuda(), bys[i].cuda())

        advs.append(advs_batch)

    torch.cuda.empty_cache()
    
    return advs

test_dev = detector.all_test_deviations

average_results = {}
for i in range(1,2):
    random.seed(i)

    validation_indices = random.sample(range(len(test_dev)),int(0.1*len(test_dev)))
    test_indices = sorted(list(set(range(len(test_dev)))-set(validation_indices)))

    validation = test_dev[validation_indices]
    test_deviations = test_dev[test_indices]

    t95 = validation.mean(axis=0)+10**-7
    if not normalize:
        t95 = np.ones_like(t95)
    
    test_deviations = (test_deviations/t95[np.newaxis,:]).sum(axis=1)
    
    torch_t95 = torch.unsqueeze(torch.tensor(t95), dim=0).cuda()

In [16]:
np.mean(test_deviations)

666.8589

In [7]:
def get_b_adv(d):
    batch_size = 32
    bx = []
    by = []
    tens = transforms.ToTensor()
    for idx in range(0,len(d),batch_size):
        bx_batch = torch.squeeze(torch.stack([tens(x[0]) for x in d[idx:idx+batch_size]]),dim=1)
        bx.append(bx_batch)
        by.append(torch.Tensor([x[1] for x in d[idx:idx+batch_size] if x[1] == 0]).type(torch.LongTensor))
        break
    
    return bx, by

In [ ]:
def bicubic_cifar(arr):
    R = arr[0:1024].reshape(32,32)
    G = arr[1024:2048].reshape(32,32)
    B = arr[2048:].reshape(32,32)
    img = np.dstack((R,G,B))
    
    fig = plt.figure(figsize=(2,2))
    ax = fig.add_subplot(111)
    ax.imshow(img)
#     title = re.sub('[!@#$b]', '', str(labels[tr_y[ind]]))
#     ax.set_title('Category = '+ title,fontsize =15)

In [ ]:
bicubic_cifar(advs_inf[0].cpu()[2].flatten())
bicubic_cifar(nxs[0].cpu()[2].flatten())
bicubic_cifar(nadvs_inf[0].detach().cpu()[2].flatten())

In [75]:
mns = [cuda(detector.mins[i]) for i in range(10)]
mxs = [cuda(detector.maxs[i]) for i in range(10)]
nxs, nys = get_b_adv(cifar10)
pinf = PGD_Tuned(mean=[0.5, .5, .5], std=[.5, .5, .5])
nadvs_inf = advs_p(pinf, nxs, nys)

Loss Step 0: tensor(-18.9446, grad_fn=<CopyBackwards>)
Loss Step 5: tensor(-45.1552, grad_fn=<CopyBackwards>)


In [76]:
adversarial_acc(nadvs_inf, nys)
adversarial_scores(nadvs_inf, ys, powers)

Adversarial Test Accuracy:  0.5555555555555556
Gram
 TNR    AUROC  DTACC  AUIN   AUOUT 
  0.000 25.159 50.000 98.381  0.259


In [50]:
adversarial_acc(nadvs_inf, nys)
adversarial_scores(nadvs_inf, ys, powers)

Adversarial Test Accuracy:  0.7222222222222222
Gram
 TNR    AUROC  DTACC  AUIN   AUOUT 
 16.667 45.857 61.411 98.784  0.590


In [33]:
adversarial_acc(nadvs_inf, nys)
adversarial_scores(nadvs_inf, ys, powers)

Adversarial Test Accuracy:  0.08333333333333333
Gram
 TNR    AUROC  DTACC  AUIN   AUOUT 
  0.000  4.648 50.000 97.887  0.202


In [20]:
def G_p_gpu(ob, p):
    temp = ob
    temp = temp.reshape(temp.shape[0],temp.shape[1],-1)
    
    temp = ((torch.matmul(temp,temp.transpose(dim0=2,dim1=1))))
    
    temp = temp.sum(dim=2)
#     temp = temp.reshape(temp.shape[0],-1)
    
    return temp.reshape(ob.shape[0], -1)

In [74]:
def normalize_l2(x):
    """
    Expects x.shape == [N, C, H, W]
    """
    norm = torch.norm(x.view(x.size(0), -1), p=2, dim=1)
    norm = norm.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
    return x / norm

class PGD_Tuned(nn.Module):
    def __init__(self, epsilon=8/255, num_steps=7, step_size=2/255, gram_weight=1, num_layers=73, grad_sign=True, mean = None, std = None):
        super().__init__()
        self.epsilon = epsilon
        self.num_steps = num_steps
        self.step_size = step_size
        self.grad_sign = grad_sign
        self.gram_weight = gram_weight
        self.num_layers = num_layers
        
        if mean is None:
            self.mean = torch.FloatTensor([0.4914, 0.4822, 0.4465]).view(1,3,1,1).cuda()
        else:
            self.mean = torch.FloatTensor(mean).view(1,3,1,1).cuda()
        if std is None:
            self.std = torch.FloatTensor([0.2023, 0.1994, 0.2010]).view(1,3,1,1).cuda()
        else:
            self.std = torch.FloatTensor(std).view(1,3,1,1).cuda()

    def get_deviation(self, feat_list, idx, mins, maxs, power=powers):
        batch_deviations = []
        for L,feat_L in enumerate(feat_list):
            if L >= self.num_layers:
                continue
                
            dev = 0
            for p,P in enumerate(power):
                g_p = G_p_gpu(feat_L,P)[idx]
                
                dev +=  (F.relu(mins[L][p]-g_p)/torch.abs(mins[L][p]+10**-6)).sum(dim=1,keepdim=True)
                dev +=  (F.relu(g_p-maxs[L][p])/torch.abs(maxs[L][p]+10**-6)).sum(dim=1,keepdim=True)
                
                batch_deviations.append(dev)
                
        return batch_deviations
        
    def gram_loss(self, feats, logits):
        confs = F.softmax(logits)
        
        _, indices = torch.max(confs, 1)
        
        loss = 0
        for i in range(10):
            idxs = indices == i

            if idxs.sum() == 0:
                continue
            
            batch_dev = self.get_deviation(feats, idxs, mins=mns[i], maxs=mxs[i])
            batch_dev = torch.squeeze(torch.stack(batch_dev, dim=1)).reshape(-1, self.num_layers)
            batch_dev /= torch_t95
            
            loss += F.relu(batch_dev).sum()
        
        return abs(loss - 10000)
    
    def forward(self, model, bx, by):
        """
        :param model: the classifier's forward method
        :param bx: batch of images
        :param by: true labels
        :return: perturbed batch of images
        """
        adv_bx = bx.detach()
        adv_bx += torch.zeros_like(adv_bx).uniform_(-self.epsilon, self.epsilon)

        for i in range(self.num_steps):
            adv_bx.requires_grad_()
            with torch.enable_grad():
                logits, feats = model.gram_forward((adv_bx - self.mean)/self.std)
                
                loss = F.cross_entropy(logits, by, reduction='sum')
                gram_loss = self.gram_loss(feats, logits)

                self.grad_sign = False
                w, gw = 0.5, 0.5
                total_loss = -1 * loss + .1 * gram_loss
                
                if i % 5 == 0:
                    print("Loss Step {}:".format(i), total_loss.cpu())
                                
            grad = torch.autograd.grad(total_loss, adv_bx, only_inputs=True)[0]            
            
            adv_bx = adv_bx.detach() - self.step_size * torch.sign(grad.detach())

            adv_bx = torch.min(torch.max(adv_bx, bx - self.epsilon), bx + self.epsilon).clamp(0, 1)

        return adv_bx

In [ ]:
# print("\n**iSUN:**\n")
# print("```")
# isun_results = detector.compute_ood_deviations(isun,POWERS=range(1,5), folder="~/gram_screenshots/isun/")
# print("```")

# print("\n**LSUN (R):**\n")
# print("```")
# lsunr_results = detector.compute_ood_deviations(lsun_r,POWERS=range(1,5), folder="~/gram_screenshots/lsun/")
# print("```")

# print("\n**LSUN (C):**\n")
# print("```")
# lsunc_results = detector.compute_ood_deviations(lsun_c,POWERS=range(1,5), folder="~/gram_screenshots/isun_c/")
# print("```")

# print("\n**TinyImgNet (R):**\n")
# print("```")
# timr_results = detector.compute_ood_deviations(tinyimagenet_r,POWERS=range(1,5), folder="~/gram_screenshots/tiny_imagnet_r/")
# print("```")

# print("\n**TinyImgNet (C):**\n")
# print("```")
# timc_results = detector.compute_ood_deviations(tinyimagenet_c,POWERS=range(1,5), folder="~/gram_screenshots/tiny_imagenet_c/")
# print("```")

# print("\n**SVHN:**\n")
# print("```")
# svhn_results = detector.compute_ood_deviations(svhn,POWERS=range(1,5), folder="~/gram_screenshots/svhn/")
# print("```")

# print("\n**CIFAR-100:**\n")
# print("```")
# c100_results = detector.compute_ood_deviations(cifar100,POWERS=range(1,11), folder="~/gram_screenshots/cifar_100/")
# print("```")

In [ ]:
#     gram_reduced = gram.sum(dim=2)
#     final = (gram_reduced.sign()*torch.abs(gram_reduced)**(1/p)).reshape(gram_reduced.shape[0],-1)
#     print(final.shape)
#     print(gram.triu().reshape(gram_reduced.shape[0],-1).shape)
    
#     mx = gram.reshape(gram_reduced.shape[0],-1).max(dim=1,keepdim=True)[0]
#     mn = gram.reshape(gram_reduced.shape[0],-1).min(dim=1,keepdim=True)[0]
#     final = mx-mn
    
    
#     if save_path != "":
#         reshaped_norm = reshaped / reshaped.norm(p=2, dim=2).unsqueeze(-1)
#         gram_norm = ((torch.matmul(reshaped_norm,reshaped_norm.transpose(dim0=2,dim1=1))))
#         gram_norm_reduced = gram_norm.sum(dim=2)
        
#         figures = {
#             "Original Gram Ch 0": gram[0].cpu(),
#             "Original Gram Reduced All Channels": gram_reduced.cpu(),
#             "Original Final All Channels": final.cpu(),
#             "Gram Normalized Features Ch 0": gram_norm[0].cpu(),
#             "Gram Normalized Reduced All Channels": gram_norm_reduced.cpu(),
#         }
        
#         fig, axs = plt.subplots(5, figsize=(6, 20))
        
#         for i, t in enumerate(figures.keys()):
#             img = axs[i].imshow(figures[t].cpu())
#             axs[i].set_title(t)
#             fig.colorbar(img, ax=axs[i])
        
#         fig.savefig(os.path.expanduser(save_path))
#         plt.close(fig)